# Demo - Answerability Intrinsic

This notebook demonstrates some examples of using the Granite answerability intrinsic. It uses the shared IO processing code for intrinsics when performing model inference with an OpenAI-compatible backend such as vLLM.

In [ ]:
# Imports go in this cell
import openai
import granite_common
from granite_common import ChatCompletion
from granite_common.base.types import UserMessage
from granite_common.intrinsics.constants import BASE_MODEL_TO_CANONICAL_NAME

## Constants

In [ ]:
intrinsic_name = "answerability"
base_model_name = "ibm-granite/granite-3.3-8b-instruct"

# Change the following two constants as needed to reflect the location of the
# inference server.
openai_base_url = "http://localhost:55555/v1"
openai_api_key = "rag_intrinsics_1234"

## Instantiate IO processing classes

In [ ]:
# Fetch IO configuration file from Hugging Face Hub
io_yaml_file = granite_common.intrinsics.util.obtain_io_yaml(
    intrinsic_name, BASE_MODEL_TO_CANONICAL_NAME[base_model_name]
)

print(
    f"Instantiating input and output processing from configuration file:\n"
    f"{io_yaml_file}"
)

intrinsics_rewriter = granite_common.IntrinsicsRewriter(config_file=io_yaml_file)
intrinsincs_result_processor = granite_common.IntrinsicsResultProcessor(
    config_file=io_yaml_file
)

## Perform input processing

In [ ]:
# Create an example chat completion with a user question and two documents.
chat_input = ChatCompletion.model_validate(
    {
        "messages": [
            {"role": "assistant", "content": "Welcome to pet questions!"},
            {"role": "user", "content": "Does my dog have fleas?"},
        ],
        "extra_body": {
            "documents": [
                {"doc_id": "1", "text": "My dog has fleas."},
                {"doc_id": "2", "text": "My cat does not have fleas."},
            ],
        },
        "model": base_model_name,
        "temperature": 0.0,
    }
)
print(chat_input.model_dump_json(indent=2))

In [ ]:
# Run request through input processing
intrinsics_input = chat_input.model_copy(deep=True)
intrinsics_input.model = intrinsic_name

intrinsics_request = intrinsics_rewriter.transform(intrinsics_input)
print(intrinsics_request.model_dump_json(indent=2))

## Run inference

In [ ]:
# Connect to the inference server
client = openai.OpenAI(base_url=openai_base_url, api_key=openai_api_key)

In [ ]:
# Pass our rewritten request directly to `chat.completions.create()`
intrinsics_completion = client.chat.completions.create(
    **intrinsics_request.model_dump()
)

print(intrinsics_request.messages[-1].content)
print(intrinsics_completion.choices[0].message.content)

## Post-process inference results

In [ ]:
processed_chat_completion = intrinsincs_result_processor.transform(
    intrinsics_completion, intrinsics_request
)

print("After post-processing, first completion is:")
print(processed_chat_completion.choices[0].model_dump_json(indent=2))

## Variations

In [ ]:
# Try some variations on the original question
variations = [
    "Does my cat have no fleas?",  # Answerable
    "Does my cat have green eyes?",  # Unanswerable
    "Does my elephant have fleas?",  # Unanswerable
    "Which of my pets have fleas?",  # Answerable
    "What is the population of Australia?",  # Unanswerable
]

for variation in variations:
    updated_request = intrinsics_request.model_copy(deep=True)
    updated_request.messages[-1] = UserMessage(content=variation)
    chat_completion = client.chat.completions.create(**updated_request.model_dump())
    print(f"'{variation}' => {chat_completion.choices[0].message.content}")